In [1]:
using Pkg
Pkg.activate(".")

using JuMP
using DataFrames
using CSV
using Gurobi
using DataFrames
using Random

include("./parameters.jl")
include("./get_functions.jl")
include("./modification_functions.jl")

construct_ef_model (generic function with 1 method)

In [7]:
loaddis = load_distribution_dict(loadcsv);

ptdfdf = DataFrame(CSV.File("./ptdfsmall.csv"));
ptdfdict = Dict()

for i = 1:38
    br = ptdfdf[i,1]
    ptdfdict[br] = Dict()
    for j = 2:25
        bus = parse(Int64,names(ptdfdf)[j])
        ptdfdict[br][bus] = ptdfdf[i,j]
    end
end

In [150]:
loadraw = 1.35*2850*ones(24);
for i = 1:3
   loadraw[i] = 0
end
windraw = zeros(24);


load = 0.01*loadraw
wind = 0.01*windraw
load

24-element Vector{Float64}:
  0.0
  0.0
  0.0
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001
 38.47500000000001

In [151]:
submodel = JuMP.read_from_file("./storage_expansion_revised/second_stage/noint_PR_scen_1.mps")

A JuMP Model
Minimization problem with:
Variables: 4416
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 4944 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3792 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 48 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [152]:
JuMP.set_optimizer(submodel, Gurobi.Optimizer)
JuMP.set_optimizer_attribute(submodel, "Method", 1) 

Academic license - for non-commercial use only - expires 2022-08-19


1

In [153]:
#adjust load balance values (remember to change load)

for bus in buses
    lf = loaddis[bus]
    for ts in timesteps
        con = get_load_balance(submodel, bus, ts)
        oldval = JuMP.constraint_object(con).set.value
        lval = load[ts]
        JuMP.set_normalized_rhs(con, lf*lval)
        newval = JuMP.constraint_object(con).set.value
        println("$(name(con)), $(oldval), $(newval)")
    end
end

Dbal_{101,1}, 0.5039153905303843, 0.0
Dbal_{101,2}, 0.5042761030571504, 0.0
Dbal_{101,3}, 0.5123921349349673, 0.0
Dbal_{101,4}, 0.5300670479791391, 1.4580000060750002
Dbal_{101,5}, 0.5830917886463912, 1.4580000060750002
Dbal_{101,6}, 0.6583003493260761, 1.4580000060750002
Dbal_{101,7}, 0.6891412694693166, 1.4580000060750002
Dbal_{101,8}, 0.6678592306459047, 1.4580000060750002
Dbal_{101,9}, 0.6305254850208737, 1.4580000060750002
Dbal_{101,10}, 0.6058166775768677, 1.4580000060750002
Dbal_{101,11}, 0.5899453263991582, 1.4580000060750002
Dbal_{101,12}, 0.5776811010006888, 1.4580000060750002
Dbal_{101,13}, 0.5666793690622167, 1.4580000060750002
Dbal_{101,14}, 0.5596454746623825, 1.4580000060750002
Dbal_{101,15}, 0.5563990621772769, 1.4580000060750002
Dbal_{101,16}, 0.5583829812023853, 1.4580000060750002
Dbal_{101,17}, 0.5987827831770336, 1.4580000060750002
Dbal_{101,18}, 0.6548735801939031, 1.4580000060750002
Dbal_{101,19}, 0.6476593296585805, 1.4580000060750002
Dbal_{101,20}, 0.63701831050

Dbal_{108,17}, 0.9480727360800001, 2.3085000000000004
Dbal_{108,18}, 1.03688316432, 2.3085000000000004
Dbal_{108,19}, 1.02546060102, 2.3085000000000004
Dbal_{108,20}, 1.00861232076, 2.3085000000000004
Dbal_{108,21}, 0.9757724518800001, 2.3085000000000004
Dbal_{108,22}, 0.90523812492, 2.3085000000000004
Dbal_{108,23}, 0.8352749257200001, 2.3085000000000004
Dbal_{108,24}, 0.8027206209630001, 2.3085000000000004
Dbal_{109,1}, 0.8165295676196662, 0.0
Dbal_{109,2}, 0.817114055509995, 0.0
Dbal_{109,3}, 0.8302650330838417, 0.0
Dbal_{109,4}, 0.8589049384665343, 2.3625000087750005
Dbal_{109,5}, 0.9448246571014541, 2.3625000087750005
Dbal_{109,6}, 1.0666903803698877, 2.3625000087750005
Dbal_{109,7}, 1.1166640935423464, 2.3625000087750005
Dbal_{109,8}, 1.082179308427418, 2.3625000087750005
Dbal_{109,9}, 1.02168481322904, 2.3625000087750005
Dbal_{109,10}, 0.981647393777698, 2.3625000087750005
Dbal_{109,11}, 0.9559299266032286, 2.3625000087750005
Dbal_{109,12}, 0.936057339160995, 2.3625000087750005


Dbal_{118,1}, 1.5537391108286929, 0.0
Dbal_{118,2}, 1.5548513077790826, 0.0
Dbal_{118,3}, 1.5798757392417215, 0.0
Dbal_{118,4}, 1.6343733874447686, 4.495499989875001
Dbal_{118,5}, 1.797866336786015, 4.495499989875001
Dbal_{118,6}, 2.029759397393207, 4.495499989875001
Dbal_{118,7}, 2.124852233891139, 4.495499989875001
Dbal_{118,8}, 2.0592326146068256, 4.495499989875001
Dbal_{118,9}, 1.9441202330018774, 4.495499989875001
Dbal_{118,10}, 1.8679347438718874, 4.495499989875001
Dbal_{118,11}, 1.8189980780547368, 4.495499989875001
Dbal_{118,12}, 1.7811833833188522, 4.495499989875001
Dbal_{118,13}, 1.7472613767263057, 4.495499989875001
Dbal_{118,14}, 1.7255735357993627, 4.495499989875001
Dbal_{118,15}, 1.715563764034539, 4.495499989875001
Dbal_{118,16}, 1.7216808476560248, 4.495499989875001
Dbal_{118,17}, 1.8462469029449444, 4.495499989875001
Dbal_{118,18}, 2.0191935259701617, 4.495499989875001
Dbal_{118,19}, 1.9969495869623661, 4.495499989875001
Dbal_{118,20}, 1.9641397781088934, 4.49549998987

In [154]:
# change ptdf constraint (remember to run load changes FIRST)
for ts in timesteps
    for br in branches
        ptdfcon = get_ptdf_con(submodel,br,ts)
        
        valold = JuMP.constraint_object(ptdfcon).set.value
        valnew = 0.0
        for bus in buses
            buscon = get_load_balance(submodel,bus,ts)
            
            loadcon = copy(JuMP.constraint_object(buscon).func)
            loadval = copy(JuMP.constraint_object(buscon).set.value)

            valnew -= ptdfdict[br][bus]*loadval

        end 

        JuMP.set_normalized_rhs(ptdfcon, valnew)
        println("$(JuMP.name(ptdfcon)), $(valold), $(valnew)")
        
    end
end


PTDF_{A1,1}, -0.07497231418358113, 0.0
PTDF_{A2,1}, -0.025998384331734195, 0.0
PTDF_{A3,1}, -0.4029446898640967, 0.0
PTDF_{A4,1}, -0.2939499835450399, 0.0
PTDF_{A5,1}, -0.23361300618452513, 0.0
PTDF_{A6,1}, -0.83508433929121, 0.0
PTDF_{A7,1}, -0.030773028702591625, 0.0
PTDF_{A8,1}, -0.6392253385519122, 0.0
PTDF_{A9,1}, -0.7342224028910451, 0.0
PTDF_{A10,1}, -0.8681731216830385, 0.0
PTDF_{A11,1}, -0.5832354016432566, 0.0
PTDF_{A12-1,1}, -0.7087289752623808, 0.0
PTDF_{A13-2,1}, -0.6723724580628759, 0.0
PTDF_{A14,1}, -1.2865860693933344, 0.0
PTDF_{A15,1}, -1.7129821550085507, 0.0
PTDF_{A16,1}, -1.3759588569926011, 0.0
PTDF_{A17,1}, -1.8086563614836146, 0.0
PTDF_{A18,1}, -4.919257239049889, 0.0
PTDF_{A19,1}, 2.2567123130092286, 0.0
PTDF_{A20,1}, -4.150678296303676, 0.0
PTDF_{A21,1}, 0.6290397791302917, 0.0
PTDF_{A22,1}, 2.991372606271666, 0.0
PTDF_{A23,1}, 1.3515309729567406, 0.0
PTDF_{A24,1}, -1.8790772180082398, 0.0
PTDF_{A25-1,1}, 0.184609607166473, 0.0
PTDF_{A25-2,1}, 0.184609607166473

PTDF_{A10,6}, -1.134156011146863, -2.5119225180952265
PTDF_{A11,6}, -0.7619216953008299, -1.6874999952750003
PTDF_{A12-1,6}, -0.9258628348335899, -2.0505959326832146
PTDF_{A13-2,6}, -0.8783677425572401, -1.9454040625917863
PTDF_{A14,6}, -1.6807584662739663, -3.7225346402242616
PTDF_{A15,6}, -2.237789859612295, -4.956244717550936
PTDF_{A16,6}, -1.797512465858932, -3.981120758670696
PTDF_{A17,6}, -2.3627758487834907, -5.233063001420066
PTDF_{A18,6}, -6.426373990052026, -14.233097895403068
PTDF_{A19,6}, 2.9481030583701853, 6.529442497507111
PTDF_{A20,6}, -5.422324905617502, -12.009335485579369
PTDF_{A21,6}, 0.8217591963317902, 1.8200277646373693
PTDF_{A22,6}, 3.9078418097141947, 8.655066624431072
PTDF_{A23,6}, 1.765600591571508, 3.9104425143821095
PTDF_{A24,6}, -2.454771599104213, -5.4368146850760155
PTDF_{A25-1,6}, 0.24116859927363365, 0.5341388920209844
PTDF_{A25-2,6}, 0.24116859927363365, 0.5341388920209844
PTDF_{A26,6}, 0.04020098596991708, 0.08903692341704605
PTDF_{A27,6}, 1.54742220

PTDF_{A20,10}, -4.990024481114729, -12.009335485579369
PTDF_{A21,10}, 0.7562436000521839, 1.8200277646373693
PTDF_{A22,10}, 3.5962851061535215, 8.655066624431072
PTDF_{A23,10}, 1.6248362702657217, 3.9104425143821095
PTDF_{A24,10}, -2.2590624111043014, -5.4368146850760155
PTDF_{A25-1,10}, 0.22194118489742748, 0.5341388920209844
PTDF_{A25-2,10}, 0.22194118489742748, 0.5341388920209844
PTDF_{A26,10}, 0.0369959210572226, 0.08903692341704605
PTDF_{A27,10}, 1.4240523786094388, 3.4272222167410322
PTDF_{A28,10}, -2.619219878315476, -6.3035943707759365
PTDF_{A29,10}, 1.353221064725496, 3.2567547141193582
PTDF_{A30,10}, 0.07083131212819624, 0.17046749839617428
PTDF_{A31-1,10}, -0.25735683833912465, -0.6193726349078215
PTDF_{A31-2,10}, -0.25735683833912465, -0.6193726349078215
PTDF_{A32-1,10}, -1.8172618774970715, -4.373547190922969
PTDF_{A32-2,10}, -1.8172618774970715, -4.373547190922969
PTDF_{A33-1,10}, -2.1762643539863356, -5.237547196660469
PTDF_{A33-2,10}, -2.1762643539863356, -5.23754719666

PTDF_{A21,14}, 0.6986078861420796, 1.8200277646373693
PTDF_{A22,14}, 3.322200856709122, 8.655066624431072
PTDF_{A23,14}, 1.501002364871568, 3.9104425143821095
PTDF_{A24,14}, -2.0868921278483588, -5.4368146850760155
PTDF_{A25-1,14}, 0.20502634603236466, 0.5341388920209844
PTDF_{A25-2,14}, 0.20502634603236466, 0.5341388920209844
PTDF_{A26,14}, 0.03417634503469801, 0.08903692341704605
PTDF_{A27,14}, 1.3155208479216103, 3.4272222167410322
PTDF_{A28,14}, -2.4196008566618286, -6.3035943707759365
PTDF_{A29,14}, 1.250087812241424, 3.2567547141193582
PTDF_{A30,14}, 0.06543303405825115, 0.17046749839617428
PTDF_{A31-1,14}, -0.23774286063895084, -0.6193726349078215
PTDF_{A31-2,14}, -0.23774286063895084, -0.6193726349078215
PTDF_{A32-1,14}, -1.6787626086583889, -4.373547190922969
PTDF_{A32-2,14}, -1.6787626086583889, -4.373547190922969
PTDF_{A33-1,14}, -2.01040437224175, -5.237547196660469
PTDF_{A33-2,14}, -2.01040437224175, -5.237547196660469
PTDF_{A34,14}, -0.06543303405825115, -0.17046749839617

PTDF_{A26,18}, 0.03999172055186664, 0.08903692341704605
PTDF_{A27,18}, 1.5393671288378799, 3.4272222167410322
PTDF_{A28,18}, -2.831315086748925, -6.3035943707759365
PTDF_{A29,18}, 1.4628001444185215, 3.2567547141193582
PTDF_{A30,18}, 0.07656698252143802, 0.17046749839617428
PTDF_{A31-1,18}, -0.2781966894418184, -0.6193726349078215
PTDF_{A31-2,18}, -0.2781966894418184, -0.6193726349078215
PTDF_{A32-1,18}, -1.9644173491995038, -4.373547190922969
PTDF_{A32-2,18}, -1.9644173491995038, -4.373547190922969
PTDF_{A33-1,18}, -2.352490582867079, -5.237547196660469
PTDF_{A33-2,18}, -2.352490582867079, -5.237547196660469
PTDF_{A34,18}, -0.07656698252143802, -0.17046749839617428
PTDF_{A1,19}, -0.0963584753702078, -0.2169206112559231
PTDF_{A2,19}, -0.033414530464676996, -0.07522224013382878
PTDF_{A3,19}, -0.5178863210591498, -1.1658571484617481
PTDF_{A4,19}, -0.37780042616986154, -0.8504980912436978
PTDF_{A5,19}, -0.3002520776865862, -0.6759225275452253
PTDF_{A6,19}, -1.0732955840552212, -2.41618532

PTDF_{A18,23}, -5.149902441996224, -14.233097895403068
PTDF_{A19,23}, 2.362520943079169, 6.529442497507111
PTDF_{A20,23}, -4.3452877650699016, -12.009335485579369
PTDF_{A21,23}, 0.6585330543278388, 1.8200277646373693
PTDF_{A22,23}, 3.1316266544610407, 8.655066624431072
PTDF_{A23,23}, 1.4148991036312957, 3.9104425143821095
PTDF_{A24,23}, -1.9671799793070273, -5.4368146850760155
PTDF_{A25-1,23}, 0.19326524728481295, 0.5341388920209844
PTDF_{A25-2,23}, 0.19326524728481295, 0.5341388920209844
PTDF_{A26,23}, 0.03221585860667549, 0.08903692341704605
PTDF_{A27,23}, 1.2400575189578944, 3.4272222167410322
PTDF_{A28,23}, -2.2808032574480754, -6.3035943707759365
PTDF_{A29,23}, 1.178377973543126, 3.2567547141193582
PTDF_{A30,23}, 0.061679543885873364, 0.17046749839617428
PTDF_{A31-1,23}, -0.2241050169441764, -0.6193726349078215
PTDF_{A31-2,23}, -0.2241050169441764, -0.6193726349078215
PTDF_{A32-1,23}, -1.582462337028849, -4.373547190922969
PTDF_{A32-2,23}, -1.582462337028849, -4.373547190922969
PT

In [155]:
#adjust wind ub values (remember to change wind)

bus = 122
for ts in timesteps
    con = get_wind_ub(submodel, bus, ts)
    oldval = JuMP.constraint_object(con).set.upper
    wval = wind[ts]
    JuMP.set_normalized_rhs(con, wval)
    newval = JuMP.constraint_object(con).set.upper
    println("$(name(con)), $(oldval), $(newval)")
end


Wub_{122,1}, 7.132000000000001, 0.0
Wub_{122,2}, 7.128, 0.0
Wub_{122,3}, 7.084, 0.0
Wub_{122,4}, 7.107, 0.0
Wub_{122,5}, 7.014, 0.0
Wub_{122,6}, 6.825, 0.0
Wub_{122,7}, 6.147, 0.0
Wub_{122,8}, 5.1770000000000005, 0.0
Wub_{122,9}, 4.266, 0.0
Wub_{122,10}, 2.742, 0.0
Wub_{122,11}, 0.93, 0.0
Wub_{122,12}, 0.063, 0.0
Wub_{122,13}, 0.038, 0.0
Wub_{122,14}, 0.011000000000000001, 0.0
Wub_{122,15}, 0.0, 0.0
Wub_{122,16}, 0.0, 0.0
Wub_{122,17}, 0.009000000000000001, 0.0
Wub_{122,18}, 2.7630000000000003, 0.0
Wub_{122,19}, 2.7289999999999996, 0.0
Wub_{122,20}, 3.4560000000000004, 0.0
Wub_{122,21}, 4.117, 0.0
Wub_{122,22}, 3.7660000000000005, 0.0
Wub_{122,23}, 5.6129999999999995, 0.0
Wub_{122,24}, 5.684, 0.0


In [156]:
for bus in buses
    unfix(get_stored_variable(submodel,bus,1))
    unfix(get_stored_variable(submodel,bus,24))
    delete(submodel,get_storage_balance(submodel,bus,1))
end

In [157]:
optimize!(submodel)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10776 rows, 4416 columns and 119880 nonzeros
Model fingerprint: 0x31f74f8f
Coefficient statistics:
  Matrix range     [2e-04, 5e+00]
  Objective range  [5e-02, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-02, 1e+01]
Presolve removed 7155 rows and 287 columns
Presolve time: 0.07s
Presolved: 3621 rows, 4198 columns, 84088 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
    4491    3.6547713e+03   0.000000e+00   0.000000e+00      1s

Solved in 4491 iterations and 1.29 seconds
Optimal objective  3.654771262e+03

User-callback calls 4537, time in user-callback 0.00 sec


In [158]:
sum = 0.
for i=101:124
    var = get_PR_variable(submodel,i)
    println(var, " ", value(var))
    sum+= value(var)
end
57.62*sum

PR_{101} 5.066491767303984
PR_{102} 0.0
PR_{103} 2.645436388091993
PR_{104} 0.0
PR_{105} 0.0
PR_{106} 4.197926809398677
PR_{107} 0.0
PR_{108} 0.0
PR_{109} 5.647933171270607
PR_{110} 5.082169412154234
PR_{111} 0.0
PR_{112} 5.450527430688286
PR_{113} 5.4222128392139854
PR_{114} 0.0
PR_{115} 2.493640639804989
PR_{116} 1.7929884283085988
PR_{117} 0.0
PR_{118} 9.486890971764755
PR_{119} 0.0
PR_{120} 0.0
PR_{121} 0.0
PR_{122} 0.0
PR_{123} 6.275797411267224
PR_{124} 0.0


3086.243319815183

In [37]:
is_fixed(get_stored_variable(submodel,101,24))

false

In [159]:
model1 = JuMP.read_from_file("./storage_expansion_revised/first_stage/first_stage_model_PR.mps")

A JuMP Model
Minimization problem with:
Variables: 24
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 24 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [163]:
budget = get_expansion_budget(model1)

set_normalized_rhs(budget, 3500)

budget

expansion_budget : 57.62 PR_{101} + 57.62 PR_{102} + 57.62 PR_{103} + 57.62 PR_{104} + 57.62 PR_{105} + 57.62 PR_{106} + 57.62 PR_{107} + 57.62 PR_{108} + 57.62 PR_{109} + 57.62 PR_{110} + 57.62 PR_{111} + 57.62 PR_{112} + 57.62 PR_{113} + 57.62 PR_{114} + 57.62 PR_{115} + 57.62 PR_{116} + 57.62 PR_{117} + 57.62 PR_{118} + 57.62 PR_{119} + 57.62 PR_{120} + 57.62 PR_{121} + 57.62 PR_{122} + 57.62 PR_{123} + 57.62 PR_{124} <= 3500.0

In [164]:
JuMP.write_to_file(model1, "./storage_expansion_revised/first_stage/first_stage_model_PR_exp1B.mps")